In [0]:
# base
import re, pickle
import numpy as np

# tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
with open('data/raw_text.txt', 'r', encoding='utf8') as myfile:
    raw_text = myfile.read()

In [0]:
def clean_tweet(t):
    # remove quotes
    t = re.sub(r'"@.*', '', t)
    t = re.sub(r'^“.*”$', '', t)
    # remove URLs
    t = re.sub(r'https*:\/\/\S*', '', t)
    t = re.sub(r'pic\.twitter\.com\/\S*', '', t)
    # remove \n
    t = re.sub('\n', ' ', t)
    # remove extra whitespaces
    t = re.sub(r'\s+', ' ', t)
    # replace '&amp' with 'and'
    t = re.sub('&amp;', 'and', t) 
    return(t)

raw_text = clean_tweet(raw_text)

In [0]:
def prepare_tweet_clf(t):
    # clean
    t = t.lower()
    t = re.sub("'ll", ' will', t) # replace abbreviations
    t = re.sub("won't", 'will not', t)
    t = re.sub("n't", ' not', t) 
    t = re.sub(r'@[A-Za-z0-9_]+', '', t) # remove @mention
    t = re.sub(r'#[A-Za-z0-9_]+', '', t) # remove #tag
    t = re.sub(r'[^a-zA-Z ]', '', t) # remove special characters
    t = re.sub("\s\s+", " ", t) # remove multiple spaces 
    return t

raw_text = prepare_tweet_clf(raw_text)

In [7]:
raw_text

'today we express our deepest gratitude to all those who have served in our armed forces busy day planned in new york will soon be making some very important decisions on the people who will be running our government love the fact that the small groups of protesters last night have passion for our great country we will all come together and be proud just had a very open and successful presidential election now professional protesters incited by the media are protesting very unfair a fantastic day in dc met with president obama for first time really good meeting great chemistry melania liked mrs o a lot happy st birthday to the us marine corps thank you for your service such a beautiful and important evening the forgotten man and woman will never be forgotten again we will all come together as never before watching the returns at pm rt such a surreal moment to vote for my father for president of the united states make your voice heard and vote rt join my family in this incredible moveme

In [8]:
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  442943
Total Vocab:  27


In [0]:
with open('chars.txt', 'wb') as fp:
    pickle.dump(chars, fp)

In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.1, random_state=42)

print("Total Patterns: ", len(X_train))

Total Patterns:  398558


In [12]:
# reshape X to be [samples, time steps, features]
X = np.reshape(X_train, (len(X_train), seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(y_train)
# define the LSTM model
model = Sequential([
    LSTM(256, dropout=0.2, input_shape=(None, X.shape[2]), return_sequences=True),
    LSTM(256, dropout=0.2, return_sequences=True),
    LSTM(256, dropout=0.2),
    Dense(y.shape[1], activation='softmax')
])

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 256)         264192    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 27)                6

In [13]:
early_stop = [EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')]

# fit the model
model.fit(X, y, epochs=200, batch_size=128, callbacks=early_stop, validation_split=0.1)

Epoch 1/200
2803/2803 [==============================] - 124s 44ms/step - loss: 2.5124 - val_loss: 2.1875
Epoch 2/200
2803/2803 [==============================] - 123s 44ms/step - loss: 2.0310 - val_loss: 1.8248
Epoch 3/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.8358 - val_loss: 1.6758
Epoch 4/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.7248 - val_loss: 1.5829
Epoch 5/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.6510 - val_loss: 1.5273
Epoch 6/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.5943 - val_loss: 1.4857
Epoch 7/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.5487 - val_loss: 1.4597
Epoch 8/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.5121 - val_loss: 1.4165
Epoch 9/200
2803/2803 [==============================] - 123s 44ms/step - loss: 1.4818 - val_loss: 1.4814
Epoch 10/200
2803/2803 [======================

In [0]:
model.save('models/model.h5')